# Tutotial DataReader

Utilisation des objets DataReader pour la lectures de données sources spécifiques. 
En résumé, que font les DataReader:
- Prise en charge d'un input specifique (une source de donnée comme par expl otoriver ou sensea).
- Prétraitmements éventuels sur la donnée brut (expl: passer sur un pas de temps horaire).
- Construction d'un dataset en sortie

In [ ]:
import os
cwd = os.getcwd()
new_cwd = os.path.join(cwd.rsplit("SeinAcoustic_DataAnalysis")[0], "SeinAcoustic_DataAnalysis") if not cwd.endswith("SeinAcoustic_DataAnalysis") else None
if new_cwd != None:
    os.chdir(new_cwd)

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import glob

In [ ]:
def get_lines (df:pd.DataFrame, x:str, y:any, color_palette="hls"):
    traces = []
    y_list = [y] if type(y) == str else y
    colors = colors = sns.color_palette(color_palette, n_colors=len(y_list)).as_hex()
    for i in range(len(y_list)):
        y = y_list[i]
        traces.append(go.Scatter(x=df[x], y=df[y], mode='lines', name=y, line=dict(color=colors[i])))
    return traces

In [ ]:
def line_graph (df:pd.DataFrame, x:str, y:any, title:str=None, xaxis_title:str=None, yaxis_title:str=None, color_palette="hls"):
    """
    tarce a line graph from dataframe
    """
    fig = go.Figure(
        data=get_lines(df=df, x=x, y=y, color_palette=color_palette)
    )
    args_layout={}
    if title:
        args_layout["title"] = title
    if xaxis_title:
        args_layout["xaxis_title"] = xaxis_title
    if yaxis_title:
        args_layout["yaxis_title"] = yaxis_title
    if len(args_layout) > 0:
        fig.update_layout(**args_layout)
    return fig

In [ ]:
def min_max_normalize(column):
    return (column - column.min()) / (column.max() - column.min())

def normalize_acoustic (df):
    cols_to_norm = [c for c in df.columns if c != "date"] 
    df_norm = df[cols_to_norm].apply(min_max_normalize, axis=0)
    df_norm.insert(0, "date", df["date"])
    return df_norm

## 1. Descripteurs Acoustiques: 

Lecture des données d'entrée sur les descripteurs acoustiques. 
2 sources de données:
- Sensea : 11 descripteurs => note : manque de métadonnées (seul info = unité = comptage sur les données de sortie du DataReader)
- Otoriver : 21 descripteurs => unités différentes en fonction des descripteurs (métadonnée complette) - a voir pout adapter le traitmeent en fonction des unités.

In [ ]:
from DBBuilder.datareader import Otoriver_DataReader

# Initialisation de du DataReader
datareader = Otoriver_DataReader(timefreq="5h") # Choisire le pas de temps de sortie
datareader.metadata # obtenir les métadonnées disponnible sur otoriver dans l'outil DataReader

In [ ]:
# Données brut otoriver
filepath = "data/Data_SAM/Bougival/input/acoustique/otoriver/otoriver_2021-07-06_2023-12-08.xlsx" 
df_otoriver_input = pd.read_excel(filepath) 
print("Données brut pour otoriver")
df_otoriver_input

In [ ]:
df_otoriver_output = datareader.from_xslx(filepath=filepath) # Lecture du fichier à l'aide du dataReader
print("Fichier de sortie pour otoriver")
df_otoriver_output

In [ ]:
otoriver_descriptors = [c for c in df_otoriver_output.columns if c != "date"]
fig_otoriver = line_graph(df_otoriver_output, x="date", y=otoriver_descriptors, title="Evolution temporelle de l'activité acoustiques par les descripteurs d'Otoriver")
fig_otoriver.show()

In [ ]:
figs = {}
metadata_otoriver = Otoriver_DataReader.metadata
otoriver_descriptors = [c for c in df_otoriver_output.columns if c != "date"]
for unit in metadata_otoriver["unit"].unique():
    y = [c for c in otoriver_descriptors if unit in c[c.index("[")+1:c.index("]")].split(",")]
    figs[unit]=line_graph(df_otoriver_output, x="date", y=y, title=f"Evolution de l'activité acoustiques des descripteurs Otoriver (type d'unité {unit})")
    figs[unit].show()

In [ ]:
df_otoriver_output_norm = normalize_acoustic(df_otoriver_output)
fig_otoriver = line_graph(df_otoriver_output_norm, x="date", y=otoriver_descriptors, title="Evolution temporelle de l'activité acoustiques par les descripteurs d'Otoriver (avec normalisation)")
# fig_otoriver.update(connectgaps=False)
fig_otoriver.show()

## 2. Descripteurs Acoustiques de Sensea : Sensea DataReader

Permettre la lecture et le prétraitement des données sensea

In [ ]:
from DBBuilder.datareader import Sensea_DataReader

# Initialisation de du DataReader
datareader = Sensea_DataReader(timefreq="15min") # Choisire le pas de temps de sortie
datareader.metadata # obtenir les métadonnées disponnible sur sensea dans l'outil DataReader

In [ ]:
filepath_sensea = "data/Data_SAM/Bougival/input/acoustique/sensea/sensea_2021-06-16_2023-11-21.csv"
df_sensea_input = pd.read_csv(filepath_sensea)
print("Données brut pour sensea")
df_sensea_input

In [ ]:
df_sensea_output = datareader.from_dataframe(df=df_sensea_input) # Lecture du fichier à l'aide du dataReader
print("Fichier de sortie pour sensea")
df_sensea_output

In [ ]:
descriptors_sensea = [c for c in df_sensea_output.columns if c != "date"]
fig_sensea = line_graph(df_sensea_output, x="date", y=descriptors_sensea)
fig_sensea.show()

In [ ]:
df_sensea_norm = normalize_acoustic(df_sensea_output)
fig_sensea = line_graph(df_sensea_norm, x="date", y=descriptors_sensea)
fig_sensea.show()

In [ ]:
df_acoustic = pd.merge(df_otoriver_output, df_sensea_output, on="date", how="outer")
df_acoustic

In [ ]:
from DBBuilder import DBBuilder

In [ ]:
dbbuilder = DBBuilder()

In [ ]:
res = dbbuilder.build(input=df_acoustic)

## Physico-Chimie

Au niveau des données physico-chimiques:
- Physico-chime continue => horaire
- Physico-chimie ponctuelle => hebdo 